## Exploratory Analysis using Jupyter Notebook

Notebooks for exploratory analysis are standard in the industry. A Jupyter Notebook provides you a Python3 REPL and the ability to display charts and graphs in-line. GitHub also understands how to show .ipynb files correctly.

Pandas is another industry standard for data analysis. We can import it and use it to read data from our s3 bucket on AWS.

In [11]:
import pandas as pd

In [47]:
import s3fs
s3 = s3fs.S3FileSystem(anon=True)
s3.ls('twde-datalab/raw')

s3.get('twde-datalab/raw/quito_stores_sample2016-2017.csv', 
       '../data/quito_stores_sample2016-2017.csv')

#### Read from local csv file to a pandas DataFrame, an incredibly powerful object

In [48]:
data = pd.read_csv('../data/quito_stores_sample2016-2017.csv')

#### Before anything else, let's look at a small slice of the data to see what we're working with

In [49]:
data.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,cluster
0,88211471,2016-08-16,44,103520,7.0,True,Quito,Pichincha,5
1,88211472,2016-08-16,44,103665,7.0,False,Quito,Pichincha,5
2,88211473,2016-08-16,44,105574,13.0,False,Quito,Pichincha,5
3,88211474,2016-08-16,44,105575,18.0,False,Quito,Pichincha,5
4,88211475,2016-08-16,44,105577,8.0,False,Quito,Pichincha,5


#### Looking 

#### Let's get a flavor of what our data looks like by looking at the first 10 records

In [42]:
data.drop(['Unnamed: 0'], axis=1).to_csv('../data/quito_stores_sample2016-2017.csv', index=False)

In [9]:
data = s3.get_object(Bucket='twde-datalab', Key='raw/quito_stores_sample2016-2017.csv')

ClientError: An error occurred (AccessDenied) when calling the GetObject operation: Access Denied

In [ ]:
s3resource.Bucket('twde-datalab').download_file('raw/quito_stores_sample2016-2017.csv', 'quito_stores_sample2016-2017.csv')